# Custom Training StyleGan2-ADA

StyleGAN2-ADA only work with Tensorflow 1. Run the next cell before anything else to make sure we’re using TF1 and not TF2.

In [1]:
import tensorflow as tf
print(tf.__version__)

1.14.0


In [2]:
!nvidia-smi

Thu Nov 12 10:24:07 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   26C    P8     6W / 180W |     11MiB /  8119MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Clone Repo locally

Clone the original repo on our server

Next, run this cell. If you’re already installed the repo, it will skip the installation process and change into the repo’s directory. If you haven’t installed it, it will install all the files necessary.

In [3]:
import os
if os.path.isdir("stylegan2-ada"):
    %cd "stylegan2-ada"
else:
    #install script
    !git clone https://github.com/dvschultz/stylegan2-ada
    %cd stylegan2-ada
    !mkdir downloads
    !mkdir datasets

Cloning into 'stylegan2-ada'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 208 (delta 1), reused 2 (delta 0), pack-reused 201
Receiving objects: 100% (208/208), 1.16 MiB | 895.00 KiB/s, done.
Resolving deltas: 100% (105/105), done.
/home/guma/Documents/App/ML/datacrunch-stylegan2-ada/stylegan2-ada


In [ ]:
%cd "stylegan2-ada"
!git config user.name "test"
!git config user.email "test@test.com"
!git fetch origin
!git checkout origin/main -- train.py

## Download dataset

Upload your dataset as a zip file to Google drive and note the file ID

In [4]:
import gdown

# file_id = 'YOUR-FILE-ID'
file_id = '1Hv0wiw5K5L4LxDAPqMsESm2jLeav5NYD'

url = f'https://drive.google.com/uc?id={file_id}'
gdown.download(url, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1Hv0wiw5K5L4LxDAPqMsESm2jLeav5NYD
To: /home/guma/Documents/App/ML/datacrunch-stylegan2-ada/stylegan2-ada/flowers-sq-1024.zip
2.91GB [06:47, 7.13MB/s]


'flowers-sq-1024.zip'

## Convert dataset to .tfrecords

**Note: You only need to do this once per dataset. If you have already run this and are returning to conntinue training, skip these cells.**

Next we need to convert our image dataset to a format that StyleGAN2-ADA can read from. There are two options here. You can upload your dataset directly to Colab (as a zipped file), or you can upload it to Drive directly and read it from there.

In [5]:
#if you manually uploaded your dataset to Colab, unzip it
# zip_path = "YOUR-ZIP-FILE.zip"
zip_path = "flowers-sq-1024.zip"

!mkdir dataset_imgs
!unzip {zip_path} -d dataset_imgs

1024/7672796078.jpg  
  inflating: dataset_imgs/flowers-sq-1024/4777693169.jpg  
  inflating: dataset_imgs/flowers-sq-1024/31020160186.jpg  
  inflating: dataset_imgs/flowers-sq-1024/17011870005.jpg  
  inflating: dataset_imgs/flowers-sq-1024/14038240103.jpg  
  inflating: dataset_imgs/flowers-sq-1024/6814568905.jpg  
  inflating: dataset_imgs/flowers-sq-1024/4366025543.jpg  
  inflating: dataset_imgs/flowers-sq-1024/17019555006.jpg  
  inflating: dataset_imgs/flowers-sq-1024/3508445546.jpg  
  inflating: dataset_imgs/flowers-sq-1024/5500631248.jpg  
  inflating: dataset_imgs/flowers-sq-1024/7253894220.jpg  
  inflating: dataset_imgs/flowers-sq-1024/12681975973.jpg  
  inflating: dataset_imgs/flowers-sq-1024/5585247242.jpg  
  inflating: dataset_imgs/flowers-sq-1024/5411570187.jpg  
  inflating: dataset_imgs/flowers-sq-1024/16427046409.jpg  
  inflating: dataset_imgs/flowers-sq-1024/13147566103.jpg  
  inflating: dataset_imgs/flowers-sq-1024/7635980226.jpg  
  inflating: dataset_imgs/f

Now that your image dataset is uploaded, we need to convert it to the `.tfrecords` format.

Depending on the resolution of your images and how many you have, this can take a while.

In [9]:
#update this to the path to your image folder
dataset_path = "dataset_imgs/flowers-sq-1024"
#give your dataset a name
dataset_name = "flowers-sq-1024"

#you don't need to edit anything here
!python dataset_tool.py create_from_images ./datasets/{dataset_name} {dataset_path}

Loading images from "dataset_imgs/flowers-sq-1024"
Creating dataset "./datasets/flowers-sq-1024"
^C
  8%|██▊                                 | 2230/28273 [06:12<1:12:26,  5.99it/s]


## Train a custom model

We’re ready to start training! There are numerous arguments to training, what’s listed below are the most popular options. To see all the options, run the following cell.

In [ ]:
!python train.py --help

In [ ]:
#this name must EXACTLY match the dataset name you used when creating the .tfrecords file
dataset_name = "flowers-sq-1024"
#how often should the model generate samples and a .pkl file
snapshot_count = 4
#should the images be mirrored left to right?
mirrored = True
#should the images be mirrored top to bottom?
mirroredY = False
#metrics? 
metric_list = None
#augments
augs = "bg"

#
# this is the most important cell to update
#
# running it for the first time? set it to ffhq(+resolution)
# resuming? get the path to your latest .pkl file and use that
# resume_from = "/content/drive/My\ Drive/colab-sg2-ada2/stylegan2-ada/results/00008-dante1024-mirror-mirrory-11gb-gpu-bg-resumecustom/network-snapshot-000160.pkl"
resume_from = "ffhq1024"
#don't edit this unless you know what you're doing :)
!python train.py --outdir ./results --snap={snapshot_count} --cfg=11gb-gpu --data=./datasets/{dataset_name} --augpipe={augs} --mirror={mirrored} --mirrory={mirroredY} --metrics={metric_list} --resume={resume_from} --augpipe="bg"

### While it’s training...
**Once the above cell is running you should be training!**

Don’t close this tab! Colab needs to be open and running in order to continue training. Every ~15min or so a new line should get added to your output, indicated its still training. Depending on you `snapshot_count` setting you should see the results folder in your Google drive folder fill with both samples (`fakesXXXXXx.jpg`) and model weights (`network-snapshot-XXXXXX.pkl`). The samples are worth looking at while it trains but don’t get too worried about each individual sample.

If you chose a metric, you will also see scores for each snapshot. Don’t obsess over these! they are a guide, it can go up or down slightly for each snapshot. What you want to see is a gradual lowering of the score over time.

Once Colab shuts off, you can Reconnect the notebook and re-run every cell from top to bottom. Make sure you update the `resume_from` path to continue training from the latest model.